# Data Preprocessing Template

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## Encode categorical data

In [3]:
dataset = pd.read_csv('/content/sample_data/chat_dataset.csv')
y = dataset.iloc[:, -1].values
X = dataset.iloc[:, :-1].values


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

dataset.iloc[:, -1] = y_encoded

dataset.to_csv('chat_dataset_encoded.csv', index=False)


<ipython-input-3-950b03aecebb>:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset.iloc[:, -1] = y_encoded


## Split csv's into training set, test set and validation set

In [4]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/chat_dataset_encoded.csv')

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(len(train_df), len(test_df), len(validation_df))

train_df.to_csv('train_dataset.csv', index=False)
validation_df.to_csv('validation_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

408 88 88


## Importing the dataset

In [5]:
train_dataset = tf.data.experimental.make_csv_dataset(
    '/content/train_dataset.csv',
    batch_size=50,
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    shuffle=True,
    ignore_errors=True)

validation_dataset = tf.data.experimental.make_csv_dataset(
    '/content/validation_dataset.csv',
    batch_size=10,
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)

test_dataset = tf.data.experimental.make_csv_dataset(
    '/content/test_dataset.csv',
    batch_size=10,
    label_name='sentiment',
    na_value="?",
    num_epochs=1,
    ignore_errors=True)


Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


## Construct Tensorflow Hub embedding model

In [6]:
import tensorflow_hub as hub


embedding = "https://www.kaggle.com/models/google/nnlm/frameworks/TensorFlow2/variations/en-dim128-with-normalization/versions/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)


## Construct model itself

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 3)                 51        
                                                                 
Total params: 124644803 (475.48 MB)
Trainable params: 124644803 (475.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
